In [6]:
import Image

ModuleNotFoundError: No module named 'Image'

In [120]:
# img = Image.open('data.tiff')
img = Image.open('reference.tiff')

AttributeError: 'TiffImageFile' object has no attribute 'shape'

In [9]:
img.show()

In [10]:
import numpy as np
from skimage.external.tifffile import imread

In [279]:
himg = imread("data.tiff")

In [280]:
himg.shape

(900, 1000, 61)

In [281]:
himg = himg / 2**16

In [212]:
cmf = np.loadtxt("CIE1931-2deg-XYZ.csv",delimiter=",")

In [213]:
cmf = cmf[np.where(cmf[:,0] >= 400)]
cmf = cmf[::2]

In [214]:
cmf.shape

(44, 4)

In [215]:
nhimg = himg[:,:,:44]

In [235]:
sd_light_source = np.loadtxt("lamp_spectrum.csv", skiprows=1,dtype="float")
sd_light_source = sd_light_source[np.where(sd_light_source[:,0] >= 400)]
sd_light_source = sd_light_source[::20,1:2]
sd_light_source = sd_light_source[:44]
print(sd_light_source)

[[8.0381e-06]
 [4.9402e-05]
 [7.9463e-04]
 [2.5200e-03]
 [5.0425e-03]
 [6.5314e-03]
 [2.9934e-03]
 [1.6326e-03]
 [1.2086e-03]
 [1.5629e-03]
 [3.2179e-03]
 [6.2438e-03]
 [9.6562e-03]
 [1.2211e-02]
 [1.3466e-02]
 [1.4078e-02]
 [1.4304e-02]
 [1.3816e-02]
 [1.3504e-02]
 [1.3538e-02]
 [1.5654e-02]
 [2.3630e-02]
 [4.3653e-02]
 [2.2712e-02]
 [7.8853e-03]
 [4.9094e-03]
 [3.7102e-03]
 [2.7212e-03]
 [2.0964e-03]
 [1.5355e-03]
 [1.1735e-03]
 [7.9114e-04]
 [6.6069e-04]
 [2.4682e-04]
 [3.1246e-04]
 [1.4981e-04]
 [9.5595e-05]
 [0.0000e+00]
 [3.0264e-04]
 [0.0000e+00]
 [0.0000e+00]
 [0.0000e+00]
 [0.0000e+00]
 [0.0000e+00]]


In [260]:
ncmf = cmf[:,1:]
nmf_multi_ld = ncmf * sd_light_source
print(nmf_multi_ld.shape)
x = nmf_multi_ld[:,0]
y = nmf_multi_ld[:,1]
z = nmf_multi_ld[:,2]
k = 100/np.sum(y)
print(k)

(44, 3)
686.3250612373361


In [251]:
X = np.sum(x*nhimg, axis=2)
Y = np.sum(y*nhimg,axis=2)
Z = np.sum(z*nhimg, axis=2)

In [271]:
XYZ = np.stack([X,Y,Z], 2)
XYZ = XYZ * k
XYZ.shape

(900, 1000, 3)

In [272]:
# xyz_to_rgb = np.array([[0.41847, -0.15866, -0.082835],[-0.091169, 0.25243, 0.015708], [0.00092090, -0.0025498, 0.17860]] )
rgb_to_xyz = np.array([[0.4900, 0.31, 0.2],[0.17697, 0.81240, 0.010630], [0, 0.01, 0.99]])
rgb_to_xyz = 1/0.17697 * rgb_to_xyz
xyz_to_rgb = np.linalg.inv(rgb_to_xyz)
print(xyz_to_rgb)

[[ 0.41846571 -0.15866078 -0.08283493]
 [-0.09116896  0.25243144  0.01570752]
 [ 0.0009209  -0.00254981  0.17859891]]


In [273]:
rgb_img =  np.dot(XYZ, xyz_to_rgb )
rgb_img.shape

(900, 1000, 3)

In [274]:
print(rgb_img.shape)
print(np.max(rgb_img))
print(np.min(rgb_img))

(900, 1000, 3)
44.50695284630758
-5.439640471637056


In [282]:
# xyz_to_r = np.array([0.41847, -0.15866, -0.082835])
xyz_to_r = np.array([3.2406, -1.5372, -0.4986])
r = np.dot(XYZ, xyz_to_r)
print(r.shape)
xyz_to_g = np.array([-0.9689, 1.8758, 0.0415])
g = np.dot(XYZ, xyz_to_g)
xyz_to_b = np.array([0.0557, -0.204, 1.0570])
b = np.dot(XYZ, xyz_to_b)
rgb_img2 = np.stack([r,g,b],axis=2)


(900, 1000)


In [283]:
print(rgb_img2.shape)
print(np.max(rgb_img2))
print(np.min(rgb_img2))
# img =Image.fromarray(rgb_img)
# img.save("made_rgb_img.png")

(900, 1000, 3)
232.9712086283906
-5.660450856145034


In [285]:
# rgb_img2*2**8
# print(np.max(rgb_img2*2**8))
# print(np.min(rgb_img2*2**8))
img =Image.fromarray(np.uint8(rgb_img2))
img.save("made_rgb_img.png")